In [51]:
!git init
!git config --global user.name "travmcwilliams"
!git config --global user.email "travmcwilliams@gmail.com"


Reinitialized existing Git repository in /mnt/batch/tasks/shared/LS_root/mounts/clusters/carsales2/code/Users/Travis_1746387082819/CarSales/.git/


In [52]:
!git config --global --add safe.directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/carsales2/code/Users/Travis_1746387082819/CarSales


In [47]:
!git branch -M main
!git remote add origin https://github.com/travmcwilliams/CarSales.git
!git remote -v


origin	https://github.com/travmcwilliams/CarSales.git (fetch)
origin	https://github.com/travmcwilliams/CarSales.git (push)


In [53]:
!git add .
!git commit -m "Initial commit: CarSales notebook, data, CI/CD configs"


[main 634d435] Initial commit: CarSales notebook, data, CI/CD configs
 2 files changed, 58 insertions(+), 12 deletions(-)


In [54]:
!git push -u origin main


Username for 'https://github.com': 

In [44]:
!git push -u origin main


fatal: 'origin' does not appear to be a git repository
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [41]:
!git init
!git add .
!git commit -m "Initial commit"
!git remote add origin https://github.com/travmcwilliams/CarSales
!git push -u origin main


Reinitialized existing Git repository in /mnt/batch/tasks/shared/LS_root/mounts/clusters/carsales2/code/Users/Travis_1746387082819/CarSales/.git/
On branch main
nothing to commit, working tree clean
error: remote origin already exists.
fatal: unable to access 'https://github.com/<you>/CarSales.git/': The requested URL returned error: 400


In [36]:
!git --version
!git config --global user.name "Travis Williams"
!git config --global user.email "your.travmcwilliams.com"


git version 2.50.1


In [37]:
!git config --global --add safe.directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/carsales2/code/Users/Travis_1746387082819/CarSales


In [38]:
# See what will be added
!git status

# Add everything except what .gitignore excludes
!git add .

# Commit
!git commit -m "Initial commit: notebook, data, CI/CD YAMLs, and ignore rules"


On branch main
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Car_Sales_Clean.ipynb
	modified:   car_sales_clean.ipynb.amltmp

no changes added to commit (use "git add" and/or "git commit -a")
[main 825043c] Initial commit: notebook, data, CI/CD YAMLs, and ignore rules
 2 files changed, 20 insertions(+), 20 deletions(-)


In [35]:
from pathlib import Path
import pandas as pd

# Candidate absolute & relative locations
candidates = [
    # Your stated absolute path (note the leading slash for Linux abs paths)
    Path("/Users/Travis_1746387082819/CarSales/CICD-project- FullCode-2/CICD-project- FullCode/data/used_cars.csv"),
    # Workspace root (sometimes files are uploaded here)
    Path("/used_cars.csv"),
    # Next to the notebook (common pattern in projects)
    Path("data/used_cars.csv"),
    Path("used_cars.csv"),
    Path("used_cars (1).csv"),
]

CSV_PATH = next((p for p in candidates if p.exists()), None)

# If not found yet, fall back to a quick search under the current project tree
if CSV_PATH is None:
    for pat in ("**/data/used_cars.csv", "**/used_cars.csv", "**/used_cars (1).csv", "**/used_cars*.csv"):
        hits = list(Path.cwd().glob(pat))
        if hits:
            CSV_PATH = hits[0]
            break

assert CSV_PATH is not None and CSV_PATH.exists(), f"Couldn't find used_cars.csv starting from {Path.cwd()}"
print("Loading dataset from:", CSV_PATH.resolve())

df_raw = pd.read_csv(CSV_PATH)
print("Shape:", df_raw.shape)
display(df_raw.head())



Loading dataset from: /mnt/batch/tasks/shared/LS_root/mounts/clusters/carsales2/code/Users/Travis_1746387082819/CarSales/used_cars.csv
Shape: (200, 7)


,Segment,Kilometers_Driven,Mileage,Engine,Power,Seats,price
0,non-luxury segment,72000,26.60,998,58.16,5,5.51
1,non-luxury segment,41000,19.67,1582,126.20,5,16.06
2,non-luxury segment,46000,18.20,1199,88.70,5,8.61
3,non-luxury segment,87000,20.77,1248,88.76,7,11.27
4,non-luxury segment,75000,21.10,814,55.20,5,10.25


In [5]:
import numpy as np, re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# ---- required columns (note: target is lowercase 'price')
REQUIRED = {"Segment","Kilometers_Driven","Mileage","Engine","Power","Seats","price"}
missing = REQUIRED - set(df_raw.columns)
assert not missing, f"Missing columns: {missing}"

# ---- convert numeric-like strings to floats
def extract_float(x):
    if pd.isna(x): return np.nan
    if isinstance(x, (int, float)): return float(x)
    m = re.search(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", str(x))
    return float(m.group(0)) if m else np.nan

for col in ["Kilometers_Driven","Mileage","Engine","Power","Seats","price"]:
    df_raw[col] = df_raw[col].apply(extract_float).astype(float)

# ---- light sanity filters
df = df_raw.dropna(subset=["price"]).copy()
df = df[(df["price"] > 0) & (df["Kilometers_Driven"] >= 0)]
print("Cleaned shape:", df.shape)

# ---- split
num_features = ["Kilometers_Driven","Mileage","Engine","Power","Seats"]
cat_features = ["Segment"]

X = df[num_features + cat_features].copy()
y = df["price"].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("Train:", X_train.shape, "Test:", X_test.shape)

# ---- preprocessing (impute + scale nums, impute + OHE cats)
numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False)),
])
categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer([
    ("num", numeric_pipe, num_features),
    ("cat", categorical_pipe, cat_features),
], remainder="drop")

print("Preprocessor ready ✅")


Cleaned shape: (200, 7)
Train: (160, 6) Test: (40, 6)
Preprocessor ready ✅


In [6]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def compute_rmse(y_true, y_pred):
    # Prefer the new API if available
    try:
        from sklearn.metrics import root_mean_squared_error
        return root_mean_squared_error(y_true, y_pred)
    except Exception:
        # Older APIs: either use squared=False or take the sqrt manually
        try:
            return mean_squared_error(y_true, y_pred, squared=False)
        except TypeError:
            return np.sqrt(mean_squared_error(y_true, y_pred))


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

baseline = Pipeline([
    ("prep", preprocess),
    ("model", RandomForestRegressor(n_estimators=200, random_state=42)),
])

baseline.fit(X_train, y_train)
preds_base = baseline.predict(X_test)

rmse_base = compute_rmse(y_test, preds_base)
mae_base  = mean_absolute_error(y_test, preds_base)
r2_base   = r2_score(y_test, preds_base)

print(f"Baseline → RMSE {rmse_base:.3f} | MAE {mae_base:.3f} | R² {r2_base:.3f}")


Baseline → RMSE 7.883 | MAE 4.768 | R² 0.925


In [8]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "model__n_estimators": [200, 400],
    "model__max_depth": [None, 10, 20],
    "model__min_samples_split": [2, 5],
}

grid = GridSearchCV(
    estimator=baseline,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=3,
    n_jobs=-1,
    verbose=1,
)
grid.fit(X_train, y_train)

best = grid.best_estimator_
preds_best = best.predict(X_test)

rmse = compute_rmse(y_test, preds_best)
mae  = mean_absolute_error(y_test, preds_best)
r2   = r2_score(y_test, preds_best)

print("Best params:", grid.best_params_)
print(f"Best → RMSE {rmse:.3f} | MAE {mae:.3f} | R² {r2:.3f}")


Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best params: {'model__max_depth': 10, 'model__min_samples_split': 2, 'model__n_estimators': 400}
Best → RMSE 7.956 | MAE 4.822 | R² 0.924


In [9]:
MAX_RMSE = rmse_base * 1.02
assert rmse <= MAX_RMSE, f"RMSE {rmse:.3f} exceeded gate {MAX_RMSE:.3f}"
print("Performance gate passed ✅")


Performance gate passed ✅


In [10]:
# --- Register the tuned model with signature, example, and metrics ---
import mlflow, shutil
from pathlib import Path
from mlflow.models.signature import infer_signature
from mlflow.tracking import MlflowClient

mlflow.set_experiment("CarSales")
mlflow.autolog(disable=True)                              # keep off
mlflow.set_registry_uri(mlflow.get_tracking_uri())

example   = X_train.head(5)
signature = infer_signature(example, best.predict(example))

LOCAL_DIR = Path("local_model")
if LOCAL_DIR.exists():
    shutil.rmtree(LOCAL_DIR)

# Save locally first (avoids AML “logged-models” API)
import mlflow.sklearn
mlflow.sklearn.save_model(
    sk_model=best,
    path=str(LOCAL_DIR),
    signature=signature,
    input_example=example,
)

# Log artifacts into a run, then register by URI
registered_ok = False
with mlflow.start_run(run_name="register_safe") as run:
    mlflow.set_tags({
        "project": "CarSales",
        "pipeline_stage": "train",
        "framework": "sklearn",
        "target": "price",
    })
    mlflow.log_metrics({"rmse": float(rmse), "mae": float(mae), "r2": float(r2)})
    mlflow.log_artifacts(str(LOCAL_DIR), artifact_path="model")

    model_uri = f"runs:/{run.info.run_id}/model"
    print("Model artifact URI:", model_uri)

    try:
        mv = mlflow.register_model(model_uri=model_uri, name="used_cars_price_prediction_model")
        print("✅ Registration requested → used_cars_price_prediction_model")
        registered_ok = True
    except Exception as e:
        print("⚠️  Registration failed (likely RBAC). Proceeding with artifacts only.\n", e)


Successfully registered model 'used_cars_price_prediction_model'.


⚠️  Registration failed (likely RBAC). Proceeding with artifacts only.
 API request to endpoint /api/2.0/mlflow/logged-models/search failed with error code 404 != 200. Response body: ''
🏃 View run register_safe at: https://eastus.api.azureml.ms/mlflow/v2.0/subscriptions/d818e748-e334-4df7-83c3-882fcc02b8b5/resourceGroups/default_resource_group/providers/Microsoft.MachineLearningServices/workspaces/gl_az_ml/#/experiments/7d4a5f5c-a66d-4d07-862a-57041a53f968/runs/dc660ab5-16bd-4f36-89d9-85c1763e3111
🧪 View experiment at: https://eastus.api.azureml.ms/mlflow/v2.0/subscriptions/d818e748-e334-4df7-83c3-882fcc02b8b5/resourceGroups/default_resource_group/providers/Microsoft.MachineLearningServices/workspaces/gl_az_ml/#/experiments/7d4a5f5c-a66d-4d07-862a-57041a53f968


In [15]:
import time
from mlflow.tracking import MlflowClient
import mlflow
import pandas as pd

MODEL_NAME = "used_cars_price_prediction_model"
client = MlflowClient()

print("Confirming registry entry…")
rm = client.get_registered_model(MODEL_NAME)
print("✅ Registered model found:", rm.name)

# --- 1) Poll MLflow registry for versions (eventual consistency) ---
versions = []
deadline = time.time() + 60  # retry up to 60s
while time.time() < deadline:
    try:
        versions = list(client.search_model_versions(f"name='{MODEL_NAME}'"))
        if versions:
            break
    except Exception as e:
        # transient errors are fine; retry
        pass
    time.sleep(3)

if not versions:
    # Try MLflow's "latest_versions" field if populated
    try:
        if getattr(rm, "latest_versions", None):
            versions = rm.latest_versions
    except Exception:
        pass

# --- 2) Fallback to AzureML SDK v2 (optional, best-effort) ---
latest_version = None
if not versions:
    try:
        from azure.ai.ml import MLClient
        from azure.identity import DefaultAzureCredential
        # These values are already implicit in your workspace; if they differ, update them:
        SUB = "d818e748-e334-4df7-83c3-882fcc02b8b5"
        RG  = "default_resource_group"
        WS  = "gl_az_ml"
        aml_client = MLClient(DefaultAzureCredential(), SUB, RG, WS)
        vers = [int(m.version) for m in aml_client.models.list(name=MODEL_NAME)]
        if vers:
            latest_version = max(vers)
            print(f"(AzureML SDK) Found versions: {sorted(vers)}")
    except Exception as e:
        print("(AzureML SDK fallback not available):", e)

# --- 3) Resolve latest_version from what we have ---
if latest_version is None and versions:
    latest_version = max(int(v.version) for v in versions)

if latest_version is None:
    print("⚠️ Model is registered, but no versions are visible yet. This can take a minute in AML.")
    print("You can still submit the notebook & artifacts; reload this cell later to complete the check.")
else:
    print("Latest version:", latest_version)
    uri = f"models:/{MODEL_NAME}/{latest_version}"
    print("Loading:", uri)
    loaded = mlflow.pyfunc.load_model(uri)

    sample = X_test.head(10)
    out = sample.copy()
    out["y_true"] = y_test.loc[sample.index].values
    out["y_pred_loaded"] = loaded.predict(sample)
    out.to_csv("sample_loaded_preds.csv", index=False)
    display(out)
    print("✅ Wrote sample_loaded_preds.csv")


Confirming registry entry…
✅ Registered model found: used_cars_price_prediction_model
⚠️ Model is registered, but no versions are visible yet. This can take a minute in AML.
You can still submit the notebook & artifacts; reload this cell later to complete the check.


In [16]:
import time, mlflow
from mlflow.tracking import MlflowClient

MODEL_NAME = "used_cars_price_prediction_model"
client = MlflowClient()

found_version = None

# Prefer Azure ML SDK (often surfaces versions sooner)
try:
    from azure.ai.ml import MLClient
    from azure.identity import DefaultAzureCredential
    SUB, RG, WS = "d818e748-e334-4df7-83c3-882fcc02b8b5", "default_resource_group", "gl_az_ml"
    aml = MLClient(DefaultAzureCredential(), SUB, RG, WS)
    versions = [int(m.version) for m in aml.models.list(name=MODEL_NAME)]
    if versions:
        found_version = max(versions)
        print(f"AzureML SDK sees versions: {sorted(versions)} | latest={found_version}")
except Exception as e:
    print("SDK listing not available:", type(e).__name__, str(e)[:160])

# Fall back to a short MLflow poll
if found_version is None:
    deadline = time.time() + 45
    while time.time() < deadline:
        try:
            mvs = list(client.search_model_versions(f"name='{MODEL_NAME}'"))
            if mvs:
                found_version = max(int(v.version) for v in mvs)
                print(f"MLflow now sees versions | latest={found_version}")
                break
        except Exception:
            pass
        time.sleep(3)

if found_version is not None:
    uri = f"models:/{MODEL_NAME}/{found_version}"
    print("Loading:", uri)
    loaded = mlflow.pyfunc.load_model(uri)
    sample = X_test.head(10).copy()
    sample["y_true"] = y_test.loc[sample.index].values
    sample["y_pred_loaded"] = loaded.predict(sample.drop(columns=["y_true"], errors="ignore"))
    sample.to_csv("sample_loaded_preds.csv", index=False)
    display(sample)
    print("✅ Wrote sample_loaded_preds.csv")
else:
    print("No versions visible yet — use Option B below to finish via run artifact.")


No versions visible yet — use Option B below to finish via run artifact.


In [17]:
import mlflow, pandas as pd

# Find the most recent 'register_safe' run in the CarSales experiment
exp = mlflow.get_experiment_by_name("CarSales")
runs = mlflow.search_runs(
    experiment_ids=[exp.experiment_id],
    order_by=["start_time DESC"],
    max_results=20,
)
# Prefer the register_safe run if present; else use the latest run
mask = runs["tags.mlflow.runName"].fillna("").str.contains("register_safe", case=False, na=False)
row = runs[mask].head(1)
if row.empty:
    row = runs.head(1)
run_id = row["run_id"].iloc[0]

model_uri = f"runs:/{run_id}/model"
print("Loading artifact model:", model_uri)
loaded = mlflow.pyfunc.load_model(model_uri)

sample = X_test.head(10).copy()
sample["y_true"] = y_test.loc[sample.index].values
sample["y_pred_loaded"] = loaded.predict(sample.drop(columns=["y_true"], errors="ignore"))
sample.to_csv("sample_loaded_preds.csv", index=False)
display(sample)
print("✅ Wrote sample_loaded_preds.csv")


Loading artifact model: runs:/dc660ab5-16bd-4f36-89d9-85c1763e3111/model


MlflowException: API request to endpoint /api/2.0/mlflow/logged-models/search failed with error code 404 != 200. Response body: ''

In [14]:
import mlflow, shutil
from pathlib import Path

run_id = "dc660ab5-16bd-4f36-89d9-85c1763e3111"  # use the run_id you printed
model_uri = f"runs:/{run_id}/model"

dst = Path("downloaded_run_model")
if dst.exists():
    shutil.rmtree(dst)

# 1) Download the logged model artifacts to a local folder
local_path = mlflow.artifacts.download_artifacts(artifact_uri=model_uri, dst_path=str(dst))
print("Downloaded to:", local_path)

# 2) Load the model from the local path (bypasses AML’s logged-models endpoint)
try:
    loaded = mlflow.pyfunc.load_model(local_path)
except Exception:
    # some stacks prefer the sklearn flavor explicitly
    import mlflow.sklearn
    loaded = mlflow.sklearn.load_model(local_path)

# 3) Make & save a small prediction sample for grading
sample = X_test.head(10).copy()
sample["y_true"] = y_test.loc[sample.index].values
sample["y_pred_loaded"] = loaded.predict(sample.drop(columns=["y_true"], errors="ignore"))
sample.to_csv("sample_loaded_preds.csv", index=False)
display(sample)
print("✅ Wrote sample_loaded_preds.csv")


MlflowException: API request to endpoint /api/2.0/mlflow/logged-models/search failed with error code 404 != 200. Response body: ''